In [250]:
!wget https://raw.githubusercontent.com/Joseph-Rance/Computing-Soc/master/Challenges/com%20soc%20ingredients.txt

--2020-11-19 18:18:47--  https://raw.githubusercontent.com/Joseph-Rance/Computing-Soc/master/Challenges/com%20soc%20ingredients.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5627 (5.5K) [text/plain]
Saving to: ‘com soc ingredients.txt.20’

com soc ingredients 100%[===================>]   5.50K  --.-KB/s    in 0s      

2020-11-19 18:18:48 (62.7 MB/s) - ‘com soc ingredients.txt.20’ saved [5627/5627]



In [251]:
with open("com soc ingredients.txt", "r") as f:
  raw = f.read()

In [252]:
dishes = []

for i in raw.split("\n")[1:-1]:
  name, popularity, pop_change, similar, cost, like, country, ingredients, link, course, description, cals, time = i.split(", ")
  similar, ingredients = similar.split(" | "), ingredients.split(" | ")
  cost, cals, time = float(cost), int(cals), float(time)
  like = 1 if like == "Y" else (-1 if like == "N" else 0)
  dishes.append({"Name": name, "Polularity": popularity, "Popularity change": pop_change, "Similar dishes": similar, "cost": cost, "Liked by Pratyaksh": like, "Country": country, "Ingredients": ingredients, "Desc": description, "Calories": cals, "Prep time": time})

print(f"No. Dishes: {len(dishes)}, example dish: {dishes[0]}")

No. Dishes: 18, example dish: {'Name': 'pepperoni pizza', 'Polularity': '99', 'Popularity change': '0', 'Similar dishes': ['Margherita Pizza'], 'cost': 3.5, 'Liked by Pratyaksh': 1, 'Country': 'Italy', 'Ingredients': ['tomatoes', 'pepperoni', 'cheese'], 'Desc': 'Even a novice cook can master the art of pizza with our simple step-by-step guide. Bellissimo!', 'Calories': 431000, 'Prep time': 0.4}


In [253]:
def P_A(dishes, dish):

  def P_AgB(a, b, dishes):
    for i in list(dishes[a]["Similar dishes"]):
        if (dishes[b]["Name"].upper() == i.upper()):
            return 1
    else:
        for x in list(dishes[a]["Similar dishes"]):
            for k in list(dishes[b]["Similar dishes"]):
                if x.upper() == k.upper():
                    return 0.5
    return 0

  if dish["Liked by Pratyaksh"] == 1:
    return 1
  if dish["Liked by Pratyaksh"] == -1: 
    return 0

  a_dictionary = {"one": 1, "two" : 2, "three": 3}

  for i in range(len(dishes)):
    if dishes[i]["Name"] == dish["Name"]:
      dish_index = i
      del dishes[i]
      break

  prob = 0
  for i in range(len(dishes)):
    prob += P_AgB(i, dish_index, dishes.copy()) * P_A(dishes.copy(), dishes[i].copy())

  return prob

In [254]:
for i in dishes:
  i["Prob"] = P_A(dishes.copy(), i.copy())

dishes = sorted(dishes, key=lambda x : x["Prob"])[::-1][:5]

for output in dishes:
  print(f"\x1b[31m{output['Name']}\x1b[0m\n{output['Desc']}\ncost: £{output['cost']}     ingredients: {', '.join(output['Ingredients'])}       probability of liking this dish: {round(output['Prob']/sum([i['Prob'] for i in dishes]), 2)}", end="\n\n")

chow mein
A classic Chinese dish of stir-fried egg noodles with shredded chicken breast. Experiment with different fish meat or vegetables
cost: £4.0     ingredients: noodles, mangetout, spring onions       probability of liking this dish: 0.3

sweet and sour chicken
Try an authentic version of this classic Chinese takeaway dish with natural sweetness and a warm chilli flavour to spice things up
cost: £4.0     ingredients: chicken, tomatoes       probability of liking this dish: 0.3

dosa
Make your own dosa – Indian rice pancakes made from fermented batter. They take a little effort but are delicious served with aloo masala and other fillings
cost: £2.5     ingredients: lentils, rice       probability of liking this dish: 0.26

Strawberry ice cream
Use sweet strawberries whilst they’re in season to make this fabulous homemade strawberry ice cream. It's perfect on its own or as an accompaniment summer desserts
cost: £1.0     ingredients:        probability of liking this dish: 0.07

hoi

Notes from after hackathon:
1. We calculated the probability of liking each food, given the information about which food pratyaksh liked.

```
P(liking food a) = sum over other food( P(loking other food) * P(liking food a given like food b)
(or P(A) = sum_B(P(B)P(A|B)))
```

P(B) is 1 or 0 if we already know if pratyaksh likes it or not. Otherwise, it is calculated recursively as above.

P(A|B) <- 2^(-n+1), where n is the minimum number of jumps between similar foods to get from A to B (In other words, the values halves for each jump).

The final answer also needs to be divided by the sum of foods to get the probability of each food being the facourite (otherwise we would get probabilities that are larger than 1).

2. We have a slight issue that the values for P(A) and similar foods are often boolean, but with a larger dataset, we could get more discrete values.

3. We could have written the P(A|B) function recursively to allow infinite range (as it is only 2 right now).